The AWS SSO part is built upon https://github.com/benkehoe/aws-sso-util


In [3]:
import boto3
import subprocess
import json
import os
import sys
import time
import re
import random
import math
import pandas as pd
from datetime import datetime, timezone, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from openpyxl import load_workbook

sys.path.append('..')
from awsutils import *
from aws_sso_lib.sso import list_available_accounts, list_available_roles, get_boto3_session


In [2]:


start_url='https://d-9267742869.awsapps.com/start'
sso_region='us-west-2'

# check if logins.json file exists to avoid running the login process
if not os.path.exists('logins.json'):
    logins = [{'account_name': account_name, 'account_id': account_id, 'role': role} for account_id, account_name, role in 
            list_available_roles(start_url=start_url, 
                                    sso_region=sso_region, 
                                    login=True)]

    # save the result to a file
    with open('logins.json', 'w') as f:
        json.dump(logins, f, indent=2)


In [3]:
if not os.path.exists('logins-sorted.json'):
    # load logins
    with open('logins.json', 'r') as f:
        logins = json.load(f)

    # order by the account_name then the priority list
    priority = ['AWSAdministratorAccess', 
                'AWSPowerUserAccess', 
                'HighBondAdministrator',
                'HighBondSupportII',
                'HighBondSupport',
                'HighBondPowerUser',
                'HighBondAssessmentsSupport',
                'DiligentSupportAccessII', 
                'DiligentSupportAccess',
                'DiligentView', 
                'DiligentMonitoringAccess',
                'DiligentDeployAdminAccess',
                'SandboxAdministratorAccess',
                'SecurityReadOnlyAccess',
                'DataLakeProductLeadership']
    logins = sorted(logins, key=lambda x: (x['account_name'], priority.index(x['role'])))


    # save the result to a file
    with open('logins-sorted.json', 'w') as f:
        json.dump(logins, f, indent=2)

In [4]:

METRIC = 'UnblendedCost'
DATE_FROM = '2024-01-01'
DATE_TO = '2024-02-01'

def get_cost_and_usage_by_services(aws_session, start, end):
    client = aws_session.client('ce')
    cu = []
    while True:
        data = client.get_cost_and_usage(
            TimePeriod={
                'Start': start,
                'End': end
            },
            Granularity='MONTHLY',
            Metrics=[METRIC],
            GroupBy=[
                {
                    'Type': 'DIMENSION',
                    'Key': 'REGION',
                },
                {
                    'Type': 'DIMENSION',
                    'Key': 'SERVICE',
                },
            ],
        )
        cu += data['ResultsByTime']
        token = data.get('NextPageToken')
        if not token:
            break

    return cu


def get_costs_and_usages(start, end):
    start_url='https://d-9267742869.awsapps.com/start'
    sso_region='us-west-2'

    with open('logins-sorted.json', 'r') as f:
        logins = json.load(f)

    
    # keep track of accountids to avoid duplicates
    account_ids = []
    result = []
    for login in logins:

        # skip if account_id is already in the list
        if login['account_id'] in account_ids:
            continue

        print(f"Getting cost and usage for {login['account_name']}")
        
        aws_session = get_boto3_session(start_url=start_url, 
                                        sso_region=sso_region, 
                                        account_id=login['account_id'], 
                                        region='us-east-1', 
                                        role_name=login['role'], 
                                        login=True)

        try:
            cu = get_cost_and_usage_by_services (aws_session, start, end)
            cu[0]['account_name'] = login['account_name']
            result += cu
        except:
            print(f"Error getting cost and usage for {login['account_name']}")
            continue

        account_ids += [login['account_id']]
        
    return result




result = get_costs_and_usages(DATE_FROM, DATE_TO)

# print json formated result
# print(json.dumps(result, indent=2))

# create a pandas table and fill in with:
# account_name, region, service, cost
# then print the table
rows_list = []
for r in result:
    groups = r['Groups']
    for g in groups:
        region = g['Keys'][0]
        service = g['Keys'][1]
        cost = round(float(g['Metrics'][METRIC]['Amount']))
        rows_list.append({'account_name': r['account_name'], 'region': region, 'service': service, 'cost': cost})

df = pd.DataFrame(rows_list)

# print(df.head(10))
#      account_name      region                     service            cost
# 0  ACL Production    NoRegion                         Tax        11134.32
# 1  ACL Production  af-south-1              AWS CloudTrail               0
# 2  ACL Production  af-south-1                  AWS Config         784.068
# 3  ACL Production  af-south-1  AWS Key Management Service    48.958131579
# 4  ACL Production  af-south-1                  AWS Lambda  175.4665254783
# 5  ACL Production  af-south-1         AWS Secrets Manager     3.871662752
# 6  ACL Production  af-south-1            AWS Security Hub   92.8975298856
# 7  ACL Production  af-south-1          AWS Step Functions    0.6867396797
# 8  ACL Production  af-south-1         AWS Systems Manager        0.101535
# 9  ACL Production  af-south-1                     AWS WAF   298.970088264


# convert services to columns
df = df.pivot_table(index=['account_name', 'region'], columns='service', values='cost', aggfunc='sum').reset_index()
df.fillna(value=0, inplace=True)
# +-----------------+---------------+-----------------+-------------+-----------------------------------+-- ...
# | account_name    | region        | AWS CloudTrail  | AWS Lambda  | Amazon Elastic Container Service  |   ...
# +-----------------+---------------+-----------------+-------------+-----------------------------------+-- ...
# | ACL Production  | eu-central-1  | 54.0            | 2542        | 43567                             |   ...
# +-----------------+---------------+-----------------+-------------+-----------------------------------+-- ...
# | dil-audit-dev   | us-west-2     | 0.0             | 346         | 8463                              |   ...
# +-----------------+---------------+-----------------+-------------+-----------------------------------+-- ...

Getting cost and usage for ACL Production


Login with IAM Identity Center required.
Attempting to open the authorization page in your default browser.
If the browser does not open or you wish to use a different device to
authorize this request, open the following URL:

https://device.sso.us-west-2.amazonaws.com/

Then enter the code:

RVWV-JTFL

Alternatively, you may visit the following URL which will autofill the code upon loading:

https://device.sso.us-west-2.amazonaws.com/?user_code=RVWV-JTFL



Getting cost and usage for ACL Services Ltd.
Getting cost and usage for ACL Staging
Getting cost and usage for acl-deploy
Getting cost and usage for acl-monitoring
Getting cost and usage for acl-playground
Getting cost and usage for acl-staging-robots
Getting cost and usage for aws-be-dev
Getting cost and usage for aws-pc-seer-playground (root)
Error getting cost and usage for aws-pc-seer-playground (root)
Getting cost and usage for dil-3pm-dev
Getting cost and usage for dil-3pm-prod
Getting cost and usage for dil-3pm-staging
Getting cost and usage for dil-3pm-tools
Getting cost and usage for dil-3rdparty-connector-discovery-dev
Getting cost and usage for dil-3rdparty-connector-discovery-prod
Getting cost and usage for dil-3rdparty-connector-discovery-staging
Getting cost and usage for dil-3rdparty-connector-discovery-tools
Getting cost and usage for dil-accuvio-dev
Getting cost and usage for dil-accuvio-prod
Getting cost and usage for dil-accuvio-staging
Getting cost and usage for dil

# Export to CSV and Excel

In [14]:
# export to csv
df.to_csv(f'costs-{DATE_FROM.replace(".","_")}.csv', index=False)

# export df into Excel format
file_path = 'costs.xlsx'
if os.path.exists(file_path):
    writer = pd.ExcelWriter(file_path, mode='a', if_sheet_exists='replace')
else:
    writer = pd.ExcelWriter(file_path, mode='w')
with writer:
    df.to_excel(writer, index=False, sheet_name=DATE_FROM)

